In [42]:
import datetime, requests, time
import xml.etree.ElementTree as ET
import pandas as pd

In [3]:
# api calls
apiBase = 'https://www.boardgamegeek.com/xmlapi2/'
gameInfo = 'thing?id='
userInfo = 'user?name='
playsInfoByUser = 'plays?username='
playsInfoByGame = 'plays?id='

In [56]:
# retrieve API call
def get_game_metadata(gameIdList):
    """supply a list of game id's you want to retrive game metadata on.  Output is XML."""
    stringify = ','.join(gameIdList)
    response = requests.get(f'{apiBase}{gameInfo}{stringify}').content
    # add a short sleep so API doesn't throttle - we aren't making that makny calls
    time.sleep(5)
    return response

In [81]:
def listify_game_metadata(apiResponse):
    """The parses the game metadata XML into a list of lists for import"""
    gameMetadata = []
    gameCategories = []
    gameMechanisms = []
    importDate = datetime.datetime.strftime(datetime.datetime.now().date(), '%Y-%m-%d')

    # game info
    tree = ET.fromstring(apiResponse)

    # iterate through response top level 'item'
    for item in tree.findall('item'):

        # define metadata we want to collect since Type & Id are attributes at the top level we will set them here
        itemType = item.attrib['type']
        itemId = item.attrib['id']      
        itemName = None                 
        itemYearPublished = None                
        itemMinPlayers = None           
        itemMaxPlayers = None           
        itemPlayTime = None             
        itemCategories = []
        itemMechanisms = []
        itemThumbnail = None
        
        # get itemName
        names = item.findall('name')
        for name in names:
            if name.attrib['type'].lower() == 'primary':
                name = name.attrib['value']
                itemName = name
        
        # get itemYearPublished
        publishedYear = item.find('yearpublished')
        itemYearPublished = publishedYear.attrib['value']

        # get itemMinPlayers
        minPlayers = item.find('minplayers')
        itemMinPlayers = minPlayers.attrib['value']

        # get itemMaxPlayers
        maxPlayers = item.find('maxplayers')
        itemMaxPlayers = maxPlayers.attrib['value']

        # get itemPlayingTime
        playingTime = item.find('playingtime')
        itemPlayTime = playingTime.attrib['value']

        # get itemCategories
        categories = item.findall('link')
        for category in categories:
            if 'categor' in category.attrib['type'].lower():
                itemCategories.append(category.attrib['value'])

        # get itemCategories
        mechanisms = item.findall('link')
        for mechanism in mechanisms:
            if 'mechanic' in mechanism.attrib['type'].lower():
                itemMechanisms.append(mechanism.attrib['value'])

        # get itemThumbnail
        thumbnail = item.find('thumbnail')
        itemThumbnail = thumbnail.text

        gameMetadata += tuple([(itemId, itemType, itemName, itemYearPublished, None, itemMinPlayers, itemMaxPlayers,
                         itemPlayTime, itemThumbnail, importDate)])
        
        gameCategories += tuple([(itemId, x) for x in itemCategories])

        gameMechanisms += tuple([(itemId, x) for x in itemMechanisms])
        
    #return [gameMetadata]
    return [gameCategories]
    #return [gameMechanisms]

In [94]:
category_list=[]

if __name__ == "__main__":
    idList = ['174430', '161936'] #'224517']
    response = get_game_metadata(idList)
    listify = listify_game_metadata(response)
    #for i in listify:
        #print(i)
    
listify



[[('174430', 'Adventure'),
  ('174430', 'Exploration'),
  ('174430', 'Fantasy'),
  ('174430', 'Fighting'),
  ('174430', 'Miniatures'),
  ('161936', 'Environmental'),
  ('161936', 'Medical')]]

In [92]:
type(listify)

list

In [95]:
categories=pd.DataFrame(listify,columns=['1','2','3','4','5'])
categories

ValueError: 5 columns passed, passed data had 7 columns

In [80]:
categories=pd.DataFrame(listify,columns=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])

ValueError: 20 columns passed, passed data had 1 columns

In [67]:
categories

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,"(174430, Adventure)","(174430, Exploration)","(174430, Fantasy)","(174430, Fighting)","(174430, Miniatures)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,"(174430, Action Queue)","(174430, Action Retrieval)","(174430, Campaign / Battle Card Driven)","(174430, Card Play Conflict Resolution)","(174430, Communication Limits)","(174430, Cooperative Game)","(174430, Critical Hits and Failures)","(174430, Deck Construction)","(174430, Deck, Bag, and Pool Building)","(174430, Grid Movement)","(174430, Hand Management)","(174430, Hexagon Grid)","(174430, Legacy Game)","(174430, Modular Board)","(174430, Once-Per-Game Abilities)","(174430, Scenario / Mission / Campaign Game)","(174430, Simultaneous Action Selection)","(174430, Solo / Solitaire Game)","(174430, Storytelling)","(174430, Variable Player Powers)"
